# Printer Development Documentation

In [3]:
import math

### Pitch Diameter Calculations

To find the path of a belt around pulleys, we need to calculate the pitch diameter (PD) of the belt for an accurate CAD representation.

The equation for pitch diameter is as follows: $$ PD = \frac{pitch \ \cdot \ numTeeth}{\pi} $$

where pitch is the distance between the teeth of the belt, and numTeeth is the amount of teeth on the pulley.




<img src="images/belt_dimensions.jpg" alt="Belt Dimensions" width="400">

For this use case, I am using 2GT-GT3 Belts. The dimensions of the belt can be found in the table above.

In addition, a 20 tooth pulley is used. Using the formula above, the PD can be calculated.


In [5]:
numTeeth = 20
pitch = 2       # mm

PD = pitch * numTeeth / math.pi
print(PD)

12.732395447351628


Since the table does not present values for the distances to the teeth and outer edge based on the PD, they are calculated below:

In [6]:
PLD = 0.254
i = 0.63
h = 0.75

belt_teeth_distance = PLD + h
outer_distance = i - PLD

print(belt_teeth_distance)
print(outer_distance)

1.004
0.376
